In [ ]:
!pip install torch
!pip install matplotlib

In [1]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

**Step 1** load the data


In [2]:
train_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    transform = ToTensor(),
    download = True
)

100%|██████████| 26421880/26421880 [00:02<00:00, 9980160.33it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 169050.56it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3177676.49it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 21378492.07it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size = 120
train_dataloader = DataLoader(train_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

In [4]:
divice = (
    "cuda"
    if torch.cuda.is_available()
    else 'mvc'
    if torch.mvc.is_available()
    else "cpu"
)
print(f'using {divice}')
class neural_network(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.layer = nn.Sequential(
        nn.Linear(28*28, 600),
        nn.ReLU(),
        nn.Linear(600, 600),
        nn.ReLU(),
        nn.Linear(600, 600),
        nn.ReLU(),
        nn.Linear(600,10),
        nn.Softmax()
    )
  def forward(self, X):
    X = self.flatten(X)
    train = self.layer(X)
    return train

model = neural_network().to(divice)
model

using cuda


neural_network(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layer): Sequential(
    (0): Linear(in_features=784, out_features=600, bias=True)
    (1): ReLU()
    (2): Linear(in_features=600, out_features=600, bias=True)
    (3): ReLU()
    (4): Linear(in_features=600, out_features=600, bias=True)
    (5): ReLU()
    (6): Linear(in_features=600, out_features=10, bias=True)
    (7): Softmax(dim=None)
  )
)

In [5]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [6]:
def train(dataloader, model, loss, optimizer):
  size = len(dataloader)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X = X.to(divice)
    y = y.to(divice)
    pred = model.forward(X)
    loss_fn = loss(pred, y)
    loss_fn.backward()
    optimizer.step()
    optimizer.zero_grad()
    if batch % 100 == 0:
      loss_fn , current = loss_fn.item(), batch + 1 * len(X)
      print(f'Loss = {loss_fn:.3f}, [{current :.3f} / {size} ]')

In [7]:
train(train_dataloader, model, loss, optimizer)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Loss = 2.302, [120.000 / 500 ]
Loss = 2.303, [220.000 / 500 ]
Loss = 2.302, [320.000 / 500 ]
Loss = 2.302, [420.000 / 500 ]
Loss = 2.302, [520.000 / 500 ]


In [8]:
def test(dataloader, model, loss):
  size = len(dataloader)
  samples = len(dataloader.dataset)
  test_loss,correct = 0,0
  model.eval()
  with torch.no_grad():
    for batch, (X,y) in enumerate(dataloader):
      X = X.to(divice)
      y = y.to(divice)
      pred = model.forward(X)
      loss_fn = loss(pred, y)
      test_loss += loss_fn
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= size
  correct /= samples
  print(f'Loss = {test_loss:.4f}, Accuracy {correct * 100} %')

In [9]:
test(test_dataloader, model, loss)

Loss = 2.3024, Accuracy 11.4 %


In [10]:
epoches = 10
for i in range(epoches):
  print(f'Epoch = {i + 1} ---------------------------------')
  train(train_dataloader, model, loss, optimizer)
  test(test_dataloader, model, loss)

Epoch = 1 ---------------------------------
Loss = 2.302, [120.000 / 500 ]
Loss = 2.302, [220.000 / 500 ]
Loss = 2.302, [320.000 / 500 ]
Loss = 2.302, [420.000 / 500 ]
Loss = 2.302, [520.000 / 500 ]
Loss = 2.3022, Accuracy 12.07 %
Epoch = 2 ---------------------------------
Loss = 2.302, [120.000 / 500 ]
Loss = 2.302, [220.000 / 500 ]
Loss = 2.302, [320.000 / 500 ]
Loss = 2.301, [420.000 / 500 ]
Loss = 2.302, [520.000 / 500 ]
Loss = 2.3020, Accuracy 12.98 %
Epoch = 3 ---------------------------------
Loss = 2.302, [120.000 / 500 ]
Loss = 2.302, [220.000 / 500 ]
Loss = 2.302, [320.000 / 500 ]
Loss = 2.301, [420.000 / 500 ]
Loss = 2.302, [520.000 / 500 ]
Loss = 2.3019, Accuracy 14.29 %
Epoch = 4 ---------------------------------
Loss = 2.302, [120.000 / 500 ]
Loss = 2.302, [220.000 / 500 ]
Loss = 2.302, [320.000 / 500 ]
Loss = 2.301, [420.000 / 500 ]
Loss = 2.302, [520.000 / 500 ]
Loss = 2.3017, Accuracy 15.47 %
Epoch = 5 ---------------------------------
Loss = 2.302, [120.000 / 500 ]
L

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Model is saved")

Model is saved


In [ ]:
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
model.eval()
X, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  x = X.to(divice)
  pred = model.forward(x)
  predicted, actual = classes[pred.argmax(1)], classes[y]
  print(f'predicted = {predicted}, actual = {actual}')

predicted = Ankle boot, actual = Ankle boot
